In [1]:
# import libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import timedelta, datetime
import time
import matplotlib.gridspec as gridspec 
from sklearn.metrics import silhouette_score,davies_bouldin_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import matplotlib.ticker as mticker
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA # Principal Component Analysis module
from sklearn.manifold import TSNE # TSNE module
from sklearn.datasets import make_blobs
import matplotlib.cm as cm
from sklearn.preprocessing import scale
import numpy as np
import folium
from folium.plugins import FastMarkerCluster
import missingno as msno
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score


In [2]:
data = pd.read_csv("./data/flipkart_com-ecommerce_sample_1050.csv")
data.head(3)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."


In [8]:
corpus = data[['description']]
corpus.head(3)

,description
0,Key Features of Elegance Polyester Multicolor ...
1,Specifications of Sathiyas Cotton Bath Towel (...
2,Key Features of Eurospa Cotton Terry Face Towe...


In [9]:
corpus.shape

(1050, 1)

## Get main goods categories

In [10]:
categories = data[['product_category_tree']] 
categories

,product_category_tree
0,"[""Home Furnishing >> Curtains & Accessories >>..."
1,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T..."
2,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T..."
3,"[""Home Furnishing >> Bed Linen >> Bedsheets >>..."
4,"[""Home Furnishing >> Bed Linen >> Bedsheets >>..."
...,...
1045,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."
1046,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."
1047,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."
1048,"[""Baby Care >> Baby & Kids Gifts >> Stickers >..."


In [11]:
categories.iloc[1]

product_category_tree    ["Baby Care >> Baby Bath & Skin >> Baby Bath T...
Name: 1, dtype: object

In [23]:
main_category['main_cat'] = categories.product_category_tree.str.split(">>",expand=True,)[0]
main_category

0                                          ["Home Furnishing 
1                                                ["Baby Care 
2                                                ["Baby Care 
3                                          ["Home Furnishing 
4                                          ["Home Furnishing 
                                  ...                        
1046                                             ["Baby Care 
1047                                             ["Baby Care 
1048                                             ["Baby Care 
1049                                             ["Baby Care 
main_cat    0       ["Home Furnishing 
1             ["Bab...
Name: 0, Length: 1051, dtype: object

In [34]:
main_category['main_cat'] = main_category['main_cat'].str.replace(r'"', '')
main_category['main_cat'] = main_category['main_cat'].str.replace(r'[', '')
main_category['main_cat'] = main_category['main_cat'].str.replace(r']', '')

<ipython-input-34-98ee0773f790>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  main_category['main_cat'] = main_category['main_cat'].str.replace(r'[', '')
<ipython-input-34-98ee0773f790>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  main_category['main_cat'] = main_category['main_cat'].str.replace(r']', '')


In [35]:
main_category['main_cat'].unique()

array(['Home Furnishing ', 'Baby Care ', 'Watches ',
       'Home Decor & Festive Needs ', 'Kitchen & Dining ',
       'Beauty and Personal Care ', 'Computers '], dtype=object)